In [1]:
import plotly.express as plx
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import scipy.stats as st

In [2]:
DATA_PATH = "../../data/input"

# Data loading

In [3]:
# Identify QC samples for median normalization correction later on
# A subbatch is defined as the samples batch between 2 QC pools.

def subbatch(x):
    if x['order'] <= 14 :
        return 0
    if x['order'] <= 26 :
        return 1
    if x['order'] <= 38 :
        return 2
    if x['order'] <= 50 :
        return 3
    if x['order'] <= 62 :
        return 4
    if x['order'] <= 74 :
        return 5
    if x['order'] <= 86 :
        return 6
    if x['order'] <= 101 :
        return 7
    else:
        return 8

In [11]:
df_data = pd.read_csv(os.path.join(DATA_PATH, 'internship_data_matrix.csv'))
df_feature_meta = pd.read_csv(os.path.join(DATA_PATH, 'internship_feature_metadata.csv'))
df_acq = pd.read_csv(os.path.join(DATA_PATH, 'internship_acquisition_list.csv'))

# Augment dataset with metadata
df_acq['sub_batch'] = df_acq.apply(lambda x : subbatch(x), axis=1)
df_data = df_data.merge(df_acq[['sample', 'class', 'order', 'batch', 'sub_batch']], on='sample')
df_data = df_data.melt(id_vars=['sample', 'class', 'order', 'batch', 'sub_batch'],value_name='intensity', var_name='feature')
df_data['feature_number'] = df_data['feature'].apply(lambda s : int(s.split('-')[1]))

# Filter classes, batch of interest and mass range of interest
df_data = df_data[df_data['class'].isin(['QC', 'Dunn', 'French', 'LMU']) & df_data['batch'] == 1]
df_glycans = df_feature_meta[df_feature_meta['mz'] > 500]
df_data = df_data.merge(df_glycans['feature'], on='feature', how='inner')

# Data processing

## Rescale data to improve comparability

There are several ways to proceed.
* Global scaling normalization coefficient : ratio of peptide abundance to median peptide abundance measured across all samples
* Median pool normalisation : ratio of peptide abundance to median peptide abundance in the three neighbouring QCs. This approach is preferred as it acounts for intra batch effects.

In [15]:
# Compute the QC pool metabolites median intensities for each data sub_batch
pools_medians = df_data[df_data['class'] == 'QC'].groupby(['sub_batch', 'feature'])['intensity'].median().reset_index()
pools_medians = pools_medians.sort_values(['feature', 'sub_batch'])\
                             .groupby('feature')\
                             .rolling(3, min_periods=3, center=True).median().dropna()\
                             .reset_index('feature').rename(columns={'intensity' : 'med_QC_intensity'})

In [17]:
# Scale data intensities
df_data_corr = df_data[df_data['class'].isin(['Dunn', 'LMU', 'French'])].merge(pools_medians, on=['feature', 'sub_batch'])
df_data_corr['intensity_corr'] = df_data_corr['intensity'] / df_data_corr['med_QC_intensity']

## Select features which have a limited variability across samples of the same class

In [40]:
# Remove outliers in QC to compute robust CV on features using Inter quartile range 

df_QC = df_data[df_data['class'] == 'QC']
df_QC_summary = df_QC.groupby(['feature'])['intensity'].describe().reset_index()
df_QC_summary['lower_IQ'] = - 1.5 * df_QC_summary['75%'] + 2.5 * df_QC_summary['25%']
df_QC_summary['higher_IQ'] = - 1.5 * df_QC_summary['25%'] + 2.5 * df_QC_summary['75%']
print(f"Before outliers removal, there are {df_QC.shape[0]} samples.")

df_QC = df_QC.merge(df_QC_summary[['feature', 'lower_IQ', 'higher_IQ']], on='feature')
df_QC = df_QC[(df_QC['intensity'] >= df_QC['lower_IQ']) & (df_QC['intensity'] <= df_QC['higher_IQ']) ]
print(f"After outliers removal, there are {df_QC.shape[0]} samples.")

Before outliers removal, there are 2728 samples
After outliers removal, there are 2592 samples


In [42]:
# Compute CV

cv_QC = df_QC.groupby('feature')['intensity'].agg(lambda x : x.std() / x.mean() * 100).reset_index().rename(columns={'intensity' : 'CV%'})
cv_QC.describe()

,CV%
count,248.000000
mean,20.206194
std,12.235200
min,2.416975
25%,14.056512
50%,17.015310
75%,22.907125
max,99.131709


According to [Instrumental Drift in Untargeted Metabolomics: Optimizing Data Quality with Intrastudy QC Samples](https://pmc.ncbi.nlm.nih.gov/articles/PMC10222478/#sec4-metabolites-13-00665), a accepted CV threshold for biomarker discovery is 20%.

In [44]:
# Select features which exhibit limited variability in QC

selected_feats = list(cv_QC[cv_QC['CV%'] <= 20]['feature'])
print(f"At this stage, {len(selected_feats)} features were selected")

At this stage, 166 features were selected
